# Canny Non Maximum Suppresion (NMS)

Vamos a implementar partiendo de cero la etapa de **supresión de no máximos** del método de Canny.

## Python básico

In [ ]:
import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt

from matplotlib.pyplot import imshow, subplot, title, plot


def fig(w,h):
    plt.figure(figsize=(w,h))

def readrgb(file):
    return cv.cvtColor( cv.imread("../images/"+file), cv.COLOR_BGR2RGB) 

def rgb2gray(x):
    return cv.cvtColor(x,cv.COLOR_RGB2GRAY)

def gray2float(x):
    return x.astype(float) #/ 255

# para ver imágenes monocromas autoescalando el rango
def imshowg(x):
    imshow(x, 'gray')
    
def grad(x):
    gx =  cv.Sobel(x,-1,1,0)/8
    gy =  cv.Sobel(x,-1,0,1)/8
    return gx,gy

En primer lugar preparamos la operación de discretización de ángulos.

In [ ]:
cy,cx = np.mgrid[-50:50,-50:50]

In [ ]:
ga = np.arctan2(cy,cx)
imshow(ga); plt.colorbar();

In [ ]:
gad = np.round(ga / np.pi * 4) % 4
imshow(gad);

Calculamos el módulo del gradiente y su ángulo discretizado en una imagen de prueba.

In [ ]:
if False:
    x = np.zeros((500,500))
    x[100:400,100:400] = 255
    gx,gy = grad(cv.GaussianBlur(x,(0,0),20))
else:
    x   = gray2float(rgb2gray(readrgb('cube3.png')))
    gx,gy = grad(cv.GaussianBlur(x,(0,0),5))

gm = np.sqrt(gx**2+gy**2)
ga = np.arctan2(gy,gx)
gad = (np.round(ga / np.pi * 4) % 4).astype(np.uint8)

fig(16,4)
subplot(1,3,1); imshowg(x), plt.title('imagen');
subplot(1,3,2); imshowg(gm), plt.title('módulo del gradiente');
subplot(1,3,3); imshow(gad); plt.colorbar(); plt.title('código de ángulo');

Implementación usando bucles de Python:

In [ ]:
%%time

nms = gm.copy() # nms: non maximum supression

r,c = x.shape

for i in range(1,r-1):
    for j in range(1,c-1):
        if    ((gad[i,j] == 0 and (gm[i,j] < gm[i,j-1] or gm[i,j] < gm[i,j+1]))
           or  (gad[i,j] == 1 and (gm[i,j] < gm[i-1,j-1] or gm[i,j] < gm[i+1,j+1]))
           or  (gad[i,j] == 2 and (gm[i,j] < gm[i-1,j] or gm[i,j] < gm[i+1,j]))    
           or  (gad[i,j] == 3 and (gm[i,j] < gm[i-1,j+1] or gm[i,j] < gm[i+1,j-1]))):
            nms[i,j] = 0  

In [ ]:
imshow( nms , 'gray', interpolation='bicubic', );

## Numpy

Implementación usando operaciones vectorizadas de numpy:

In [ ]:
# %%timeit

G  = gm[1:-1,1:-1]
Ga = gm[1:-1,2:]
Gb = gm[:-2,2:]
Gc = gm[:-2,1:-1]
Gd = gm[:-2,:-2]
Ge = gm[1:-1,:-2]
Gf = gm[2:,:-2]
Gg = gm[2:,1:-1]
Gh = gm[2:,2:]

A = gad[1:-1,1:-1]

mask = ( (A==0) & (G > Ga) & (G > Ge) 
       | (A==1) & (G > Gd) & (G > Gh) 
       | (A==2) & (G > Gc) & (G > Gg)
       | (A==3) & (G > Gb) & (G > Gf) )

canny = np.zeros_like(G)
canny[mask]=G[mask]

In [ ]:
fig(12,8)
imshow( canny, 'gray', interpolation='bicubic');

## C

El código anterior se puede acelerar aún más si lo escribimos en un lenguaje compilado. No es complicado escribir [extensiones](https://docs.scipy.org/doc/numpy-1.13.0/user/c-info.python-as-glue.html) para manipular arrays de numpy.

Vamos a usar C plano para acceder directamente a los arrays de imagen.

La forma de crear un interfaz con C se explicará en detalle en el laboratorio. Cuando esté terminado tendremos una función *wrapper* que admite los tipos de Python.

In [ ]:
# Añadimos por programa la ubicación del nuestro módulo al path.
# (Otra posibilidad es añadir esa ruta a PYTHONPATH)

import os, sys
sys.path.append(os.getcwd()+"/../code/inC")

In [ ]:
import cfuns

In [ ]:
cnms = cfuns.nms(gm,gad)

fig(12,8)
imshow(cnms, 'gray', interpolation='bicubic');

In [ ]:
%%timeit
cnms = cfuns.nms(gm,gad)

Si recompilamos con optimización bajamos a 2ms.

Es una mejora significativa pero cuyo impacto en el rendimiento global dependerá mucho del resto de etapas de la cadena de proceso. Solo tiene sentido dedicar tiempo a optimizar las etapas más lentas.

La implementación en C no admite directamente slices, pero si es necesario hacemos una copia de las entradas.

In [ ]:
cnms = cfuns.nms(gm[::4,::4].copy(),gad[::4,::4].copy())

imshow(cnms, 'gray', interpolation='bicubic');

Comparemos el rendimiento con la implementación de OpenCV.

In [ ]:
xs = cv.GaussianBlur(x,(0,0),5).astype(np.uint8)

In [ ]:
%%timeit

cannycv = cv.Canny(xs,20,60)

Es mucho más rápido, teniendo en cuenta que tiene que calcular los gradientes y aplicar el doble umbralizado. Podemos echar un vistazo al código fuente [canny.cpp](https://github.com/opencv/opencv/blob/master/modules/imgproc/src/canny.cpp). Son más de 1000 líneas de código C++ con implementaciones alternativas dependiendo de las instrucciones disponibles en cada procesador.